In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import mutual_info_score
from sklearn.tree import DecisionTreeClassifier

####################################################################
#                          Read data                               #
####################################################################

prefix = ""

_test_x = pd.read_table(prefix + "artificial_test.data", sep=" ", header=None)
_test_x.drop(_test_x.columns[500], axis=1, inplace=True)
_train_y = pd.read_table(prefix + "artificial_train.labels", header=None)
_train_x = pd.read_table(prefix + "artificial_train.data", sep=" ", header=None)
_train_x.drop(_train_x.columns[500], axis=1, inplace=True)

In [ ]:
def get_train_and_validation_data():
    split=400
    train_x, valid_x = _train_x[:split], _train_x[split:]
    train_y, valid_y = _train_y[:split], _train_y[split:]
    return train_x, train_y, valid_x, valid_y

In [ ]:
train_x, train_y, valid_x, valid_y = get_train_and_validation_data()

In [ ]:
# 1. Remove Highly Correlated Columns
def remove_highly_correlated_features(train_x, valid_x, threshold=0.95):
    corr_matrix = train_x.corr().abs()
    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    # Find index of feature columns with correlation greater than threshold
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    train_x = train_x.drop(to_drop, axis=1)
    valid_x = valid_x.drop(to_drop, axis=1)
    return train_x, valid_x

In [ ]:
train_x, valid_x = remove_highly_correlated_features(train_x, valid_x)

In [ ]:
train_x.shape

In [ ]:
# 2. Remove Low Variance Columns
def remove_low_variance_features(train_x, valid_x, threshold=(0.8 * (1 - 0.8))):
    sel = VarianceThreshold(threshold=threshold)
    sel.fit(train_x)
    train_x = train_x[train_x.columns[sel.get_support(indices=True)]]
    valid_x = valid_x[valid_x.columns[sel.get_support(indices=True)]]
    return train_x, valid_x

In [ ]:
train_x, valid_x = remove_low_variance_features(train_x, valid_x)

In [ ]:
train_x.shape

In [ ]:
# 3. Remove Random Columns (Optional)
# This step is an approximation and should be tailored to your specific needs
# Here we use a Decision Tree to estimate feature importance
def remove_random_features(
    train_x: pd.DataFrame, train_y: pd.DataFrame, valid_x: pd.DataFrame, importance=0.001
):
    tree: DecisionTreeClassifier = DecisionTreeClassifier(random_state=0)
    tree.fit(train_x, train_y)
    importances = tree.feature_importances_

    # Assume columns with very low importance are "random"
    # This threshold can be adjusted based on domain knowledge
    important_indices = [i for i, imp in enumerate(importances) if imp > importance]
    train_x = train_x.iloc[:, important_indices]
    valid_x = valid_x.iloc[:, important_indices]
    return train_x, valid_x

In [ ]:
train_x, valid_x = remove_random_features(
    train_x=train_x, train_y=train_y, valid_x=valid_x
)

In [ ]:
train_x.shape

In [ ]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif


def anove_filter(
    train_x: pd.DataFrame,
    train_y: pd.DataFrame,
    valid_x: pd.DataFrame,
    k: int = 50,
):
    # Using ANOVA F-test to select features
    selector = SelectKBest(
        f_classif, k=k
    )  # Change k to select the number of features you want
    selector.fit(train_x, train_y)

    # Get F-values and p-values for each feature
    f_values = selector.scores_
    p_values = selector.pvalues_

    # Selecting features (you can use a threshold or select top k features)
    selected_features = train_x.columns[selector.get_support()]

    # Transforming train_x to include only the selected features
    train_x = selector.transform(train_x)
    valid_x = selector.transform(valid_x)
    return train_x, valid_x

In [ ]:
# train_x, valid_x = anove_filter(train_x=train_x, train_y=train_y, valid_x=valid_x)

In [ ]:
label = "class"
train_y = train_y.rename(columns={0: label})
valid_y = valid_y.rename(columns={0: label})

In [ ]:
train_data = pd.concat([train_x, train_y[label]], axis=1)

In [ ]:
# sanity check
(
    original_train_x,
    original_train_y,
    original_valid_x,
    original_valid_y,
) = get_train_and_validation_data()
for y, original_y in zip([train_y, valid_y], [original_train_y, original_valid_y]):
    assert y.shape == original_y.shape

In [ ]:
from autogluon.tabular import TabularPredictor

save_path = "some_path"
# trina for 3 minutes
predictor = TabularPredictor(
    label=label, path=save_path, eval_metric="balanced_accuracy"
    
).fit(train_data, time_limit=60 * 3)

In [ ]:
predictor.leaderboard()

In [ ]:
# check on validation data
valid_data = pd.concat([valid_x, valid_y[label]], axis=1)
predictor.evaluate(valid_data)
# best model WeightedEnsamble_L2 score 0.86